This is very simple customer behaviour plot on a specific Year 2020. We can try different thresholds to analyse the plots. 

In [ ]:
import pandas as pd
import numpy as np
import time
from datetime import datetime as dt
from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
# LOAD TRANSACTIONS DATAFRAME
tnx_df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
tnx_df['Year'] = tnx_df.t_dat.apply(lambda x: dt.strptime(x,'%Y-%m-%d')).dt.year
tnx_df['WK'] = tnx_df.t_dat.apply(lambda x: dt.strptime(x,'%Y-%m-%d')).dt.week
tnx_df = tnx_df.groupby(by=['customer_id','Year','WK','sales_channel_id']).price.agg(['count', 'sum']).reset_index()

tnx_df.head()

## Year 2020 Customer

In [ ]:
tnx_df = tnx_df[tnx_df.Year==2020]

## RFM Data Prep

In [ ]:
rfm= tnx_df.groupby('customer_id').agg({'WK':'max','count': sum,'sum': sum})
rfm.columns=['recency','frequency','monetary']
rfm['frequency'] = rfm['frequency'].astype(int)
rfm['recency'] = rfm['recency'].astype(int)
rfm.head()

In [ ]:
rfm['r_qtl'] = pd.qcut(rfm['recency'], 4, ['1','2','3','4'])#,'5','6','7','8','9','10'])
rfm['f_qtl'] = pd.qcut(rfm['frequency'], 4, ['1','2','3','4'])
rfm['m_qtl'] = pd.qcut(rfm['monetary'], 4, ['1','2','3','4'])
rfm['RFM_Score'] = rfm.r_qtl.astype(str)+ rfm.f_qtl.astype(str) + rfm.m_qtl.astype(str)
rfm =rfm.reset_index()
rfm.head()

## User define Threshold

In [ ]:
def cust_group(x):
    if x=='444':
        xx = 'Most Valued Customer'
    elif x == '111':
        xx = 'Lazy Customer'
    elif x in ['441','442','443']:
        xx = 'Most frequent customer'
    else:
        xx= 'Other Customer'
    return xx;

In [ ]:
rfm['Customer group'] =rfm['RFM_Score'].apply(lambda x: cust_group(x) )

rfm_plot_df = rfm.groupby('Customer group').customer_id.count().reset_index()
rfm_plot_df = rfm_plot_df[rfm_plot_df['Customer group']!='Other Customer']
plt.figure(figsize = (10,7))
ax = sns.barplot(x="Customer group", y="customer_id", data=rfm_plot_df)